In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from explorer import Explorer

Additionally, Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/faiss/loader.py:28: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(numpy.__version__) >= "1.19":
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/setuptools/_distutils/version.py:337: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to 

In [2]:
e = Explorer()
e.adaptor.initialize_env()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Products loaded.

Keys cleaned.

Attributes loaded.

100%|██████████| 1000/1000 [00:00<00:00, 126570.83it/s]


0 skipped

Loaded 13 goals.


/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)
/home/xingkun/miniconda3/envs/webshop/lib/python3.8/site-packages/gym/envs/registration.py:619: UserWarning: WARN: Env check failed with the following message: You must specify an observation space (cf gym.spaces) cf https://github.com/openai/gym/blob/master/gym/spaces/
You can set `disable_env_checker=True` to disable this check.
  logger.warn(
/home/xingkun/webshop/web_agent_site/envs/web_agent_text_env.py:212: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = self._parse_html(html).findAll(text=True)


In [3]:
e.explore()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Start exploring at 2025-11-15_16:29:51
Step 0
Current state url: http://127.0.0.1:3000/<session_id>
Action status: {'has_search_bar': True, 'clickables': ['search']}
Retrieved 2 experiences: [{'id': '11-15_14:15:24.809830_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]'], 'st': {'url': 'http://127.0.0.1:3000/<session_id>', 'html': 'WebShop Search'}, 'action': 'search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'st1': {'url': 'http://127.0.0.1:3000/search_results/<session_id>/<query>/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women's Coat, FORUU Winter Faux Fur Fleece Outwear Warm Lapel Biker Motor Aviator Jacket $21.49 to $24.99 B09MSZ4VS

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]' is taken
Step 1
Current state url: http://127.0.0.1:3000/search_results/<session_id>/<query>/1
Action status: {'has_search_bar': False, 'clickables': ['back to search', 'next >', 'b09kp78g37', 'b07zxbgdxf', 'b09msz4vs8', 'b09glvmlms', 'b09c8ynwwb', 'b09s3bn15c', 'b000j3hzxs', 'b07dkgjr74', 'b098xt346y', 'b0957xw92m']}
Retrieved 2 experiences: [{'id': '11-15_14:15:34.576029_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]'], 'st': {'url': 'http://127.0.0.1:3000/search_results/<session_id>/<query>/1', 'html': "Back to Search Page 1 (Total results: 50) Next > B09KP78G37 Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece Outwear Trench Zipper Plus Size Long Sleeve Plush Overcoat $47.41 to $59.07 B07ZXBGDXF Women'

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Action 'click[x-large]' is taken
Step 4
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 3 experiences: [{'id': '11-15_14:15:42.985567_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[x-large]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[x-large]'], 'st': {'url': 'http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}', 'html': 'Back to Search < Prev color black navy red yellow size small medium large x-large xx-large 3x-large Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[x-large]' is taken
Step 5
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 3 experiences: [{'id': '11-15_14:15:42.985567_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[x-large]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[x-large]'], 'st': {'url': 'http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large"}', 'html': 'Back to Search < Prev color black navy red yellow size small medium large x-large xx-large 3x-large Women Faux Fur Lined Jacket Coat Winter Warm Thick Fleece

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Action 'click[red]' is taken
Step 6
Current state url: http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large", "color": "red"}
Action status: {'has_search_bar': False, 'clickables': ['back to search', '< prev', 'description', 'features', 'reviews', 'buy now', 'black', 'navy', 'red', 'yellow', 'small', 'medium', 'large', 'x-large', 'xx-large', '3x-large']}
Retrieved 1 experiences: [{'id': '11-15_14:15:56.273898_9_search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]-click[b09kp78g37]-click[x-large]-click[x-large]-click[x-large]-click[red]-click[buy now]', 'action_path': ['search[x-large red women faux fur lined winter warm jacket coat price:lower than 90]', 'click[b09kp78g37]', 'click[x-large]', 'click[x-large]', 'click[x-large]', 'click[red]', 'click[buy now]'], 'st': {'url': 'http://127.0.0.1:3000/item_page/<session_id>/B09KP78G37/<query>/1/{"size": "x-large", "color": "red"}', 'html': 'Back to Search < Prev color black navy

In [4]:
# a = Explorer()
# a.adaptor.initialize_env()
# a.execute_step()